In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_parquet('output_run.parquet')

In [3]:
# 데이터 분리 (학습, 검증, 테스트)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['review'], df['voted_up'], test_size=0.2, random_state=42)

test_texts, val_texts, test_labels, val_labels = train_test_split(
    val_texts, val_labels, test_size=0.5, random_state=42)

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "beomi/KcELECTRA-small-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) # 이진분류

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-small-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

# PyTorch 데이터셋 정의
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]

        # 토큰화
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
            add_special_tokens=True,
            stride=32,
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }

In [6]:
# 데이터셋 생성
train_dataset = ReviewDataset(train_texts, train_labels, tokenizer)
val_dataset = ReviewDataset(val_texts, val_labels, tokenizer)
test_dataset = ReviewDataset(test_texts, test_labels, tokenizer)

In [8]:
from torch.utils.data import DataLoader

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [9]:
from transformers import Trainer, TrainingArguments

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./results",            # 모델 저장 경로
    eval_strategy="epoch",            # 매 에포크마다 평가
    save_strategy="epoch",            # 매 에포크마다 저장
    learning_rate=5e-5,               # 학습률
    per_device_train_batch_size=2,   # 학습 중 GPU/CPU당 처리할 배치 크기
    per_device_eval_batch_size=8,    # 평가 중 GPU/CPU당 처리할 배치 크기
    num_train_epochs=3,               # 학습할 에포크 수
    weight_decay=0.01,                # 가중치 감소 - 모델 과적합 방지
    logging_dir="./logs",             # 로그 저장 경로
    logging_steps=500,                # 몇 개의 스텝마다 로그를 기록할지
    save_total_limit=2,               # 저장할 체크포인트 개수 제한
    load_best_model_at_end=True,      # 최적의 모델 로드
    gradient_accumulation_steps=8,
    fp16=True,
)

In [10]:
# Trainer 정의
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss
0,0.471300,0.450964
1,0.382000,0.448269
2,0.335800,0.467767


TrainOutput(global_step=4371, training_loss=0.40605461698351886, metrics={'train_runtime': 891.6922, 'train_samples_per_second': 78.444, 'train_steps_per_second': 4.902, 'total_flos': 514373000306688.0, 'train_loss': 0.40605461698351886, 'epoch': 2.9994853319608854})

In [11]:
# 평가 함수
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test Results: {eval_results}")

Test Results: {'eval_loss': 0.4306432604789734, 'eval_runtime': 3.4566, 'eval_samples_per_second': 843.305, 'eval_steps_per_second': 105.594, 'epoch': 2.9994853319608854}


In [12]:
# 예측 수행 함수
def predict(texts):
    # 사용할 디바이스 설정 (GPU 사용 가능하면 CUDA, 아니면 CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 모델을 디바이스로 이동
    model.to(device)

    # 입력 데이터 토크나이징 및 디바이스 이동
    encodings = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
        add_special_tokens=True,
        stride=32,
    )
    input_ids = encodings["input_ids"].to(device)
    attention_mask = encodings["attention_mask"].to(device)

    # 모델 추론
    with torch.no_grad():  # 예측 시에는 Gradient 계산 비활성화
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

    # 예측 결과 계산
    predictions = torch.argmax(outputs.logits, dim=-1)

    # 결과를 CPU로 이동한 후 Numpy 배열로 반환
    return predictions.cpu().numpy()

In [1]:
import re

def detect_all_eng(text):
    if not isinstance(text, str):
        return ''
    
    if re.fullmatch(r"[A-Za-z\s]*", text):
        return False
    else:
        return text

In [94]:
sample = pd.read_parquet('./output_run_HFF.parquet')

In [95]:
sample['review'] = sample['review'].apply(detect_all_eng)

In [99]:
sample_texts = sample['review'].tolist()
answer = sample['voted_up'].tolist()

In [100]:
# sample_texts에서 유효하지 않은 값 제거
sample_texts = [text for text in sample_texts if isinstance(text, str) and text.strip() != '']

In [102]:
answer = [int(value) for value in answer]

In [103]:
predictions = predict(sample_texts)
print("Predictions:", predictions)  # [1, 0] -> 긍정, 부정

Predictions: [1 1 1 ... 1 1 1]


In [107]:
count = 0
eng_count = 0
for i in range(len(sample_texts)):
    if predictions[i] == answer[i]:
        count += 1
    else:
        print(f"Text: {sample_texts[i]}")
        print(f"Predict: {predictions[i]}, Answer: {answer[i]}")
        print()
        
        if not detect_all_eng(sample_texts[i]):
            print("\n\n\n\n\n\n\n\n\n\n\n\n" + sample_texts[i])
            eng_count += 1

print(f"Accuracy: {count / len(sample_texts) * 100:.2f}%")
print(f"The number of Wrong Answers: {len(sample_texts) - count}")
print(f"The number of english reviews: {eng_count}")

Text: 아니분명잡았다고왜놓냐고마우스누르고있는데놓지말라고일어서라고왜멋대로쓰러지는데
Predict: 0, Answer: 1

Text: 내 기준에선 별로였음 컨트롤도 어려워서 제대로 못하겠고 재미는 있긴 친구들이 알아서 다 하니까 음
Predict: 0, Answer: 1

Text: 주기적으로 한번씩 해야하는겜 근데 맨날 엉덩이 자랑만 하다가 끈다
Predict: 1, Answer: 0

Text: 흐물흐물
Predict: 0, Answer: 1

Text: 너무어려움 조작이
Predict: 0, Answer: 1

Text: 5천원 이하면 한다
Predict: 0, Answer: 1

Text: 평소에도 못했는 손잡는걸 이게임에서 하고있음ㅇㅇ
Predict: 0, Answer: 1

Text: 개판 오분전
Predict: 0, Answer: 1

Text: 학교 강의 시간에 하면 안되는 게임
Predict: 0, Answer: 1

Text: 멀미나요
Predict: 0, Answer: 1

Text: 놓으라고
Predict: 0, Answer: 1

Text: 개웃김
Predict: 0, Answer: 1

Text: 해저에서 마지막 기둥으로 다리만드는거 2번 안쓰러지고 체크포인트 로딩해서 3번째에서야 쓰러졌습니다 박물관에서는 헬리콥터에서 2번 고리가 떨어지지 않았고 1번은 이상한 곳에 떨어져 고리를 걸 수 없었습니다 플레이를 끝까지 다 했는데 버그때문에 깰 수가 없습니다 버그가 너무 심합니다
Predict: 0, Answer: 1

Text: 잡아땡기지마라고
Predict: 0, Answer: 1

Text: 친구들과의 열정적인 소통과 더욱 깊은 우애를 쌓고 만들어 갈 수 있는 정말 좋은 게임 입니다 정말
Predict: 1, Answer: 0

Text: s2
Predict: 0, Answer: 1

Text: 시발 좆같은 게임
Predict: 0, Answer: 1

Text: 한번쯤은 해볼만한 게임 친구없이 하면 재미 도 없는
Predict: 0,

In [108]:
import torch

torch.cuda.empty_cache()